In [ ]:
from pyspark.sql import SparkSession

ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
1,application_1662083654699_0003,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
spark = SparkSession.builder.getOrCreate()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
spark.sparkContext.getConf().get('spark.driver.memory')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

'10g'

In [4]:
output_path ="s3://patents-analytics/cleaned"

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
import pyspark.sql.types as T
import pyspark.sql.functions as F
import pyspark.sql.window as w

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
import sparknlp
from sparknlp.base import *
from sparknlp.annotator import *
from pyspark.ml import Pipeline

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Read the cleaned patent data

In [7]:
patent_cleaned_path = "s3://patents-analytics/cleaned/patent.parquet/"

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
patent_cleaned = spark.read.parquet(patent_cleaned_path)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
patent_cleaned.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- id: string (nullable = true)
 |-- type: string (nullable = true)
 |-- date: timestamp (nullable = true)
 |-- abstract: string (nullable = true)
 |-- title: string (nullable = true)
 |-- kind: string (nullable = true)
 |-- num_claims: integer (nullable = true)
 |-- year: integer (nullable = true)

In [10]:
def combine_details(abstract, title):
    details = ""
    if title:
        details= details + title
    if abstract:
        details=details + abstract
    return details

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
combine_details_udf = F.udf(combine_details, T.StringType())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
patent_cleaned_combined = patent_cleaned.withColumn("details", combine_details_udf(F.col("abstract"), F.col("title"))).select(F.col("id"), F.col("details"))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
subset = patent_cleaned_combined.limit(10).cache()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Prepare the YAKE Pipeline

In [14]:
document_assembler = DocumentAssembler().setInputCol("details").setOutputCol("document")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:
sentence_detector = SentenceDetector().setInputCols(["document"]).setOutputCol("sentence")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
token = Tokenizer().setInputCols(["sentence"]).setOutputCol("token")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [17]:
keywords = YakeKeywordExtraction().setInputCols(["token"]).setOutputCol("keywords").setMinNGrams(1).setMaxNGrams(2).setWindowSize(2).setThreshold(0.6)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [18]:
pipeline = Pipeline().setStages([document_assembler, sentence_detector, token, keywords])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [19]:
result = pipeline.fit(patent_cleaned_combined).transform(patent_cleaned_combined)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [20]:
result.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- id: string (nullable = true)
 |-- details: string (nullable = true)
 |-- document: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- annotatorType: string (nullable = true)
 |    |    |-- begin: integer (nullable = false)
 |    |    |-- end: integer (nullable = false)
 |    |    |-- result: string (nullable = true)
 |    |    |-- metadata: map (nullable = true)
 |    |    |    |-- key: string
 |    |    |    |-- value: string (valueContainsNull = true)
 |    |    |-- embeddings: array (nullable = true)
 |    |    |    |-- element: float (containsNull = false)
 |-- sentence: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- annotatorType: string (nullable = true)
 |    |    |-- begin: integer (nullable = false)
 |    |    |-- end: integer (nullable = false)
 |    |    |-- result: string (nullable = true)
 |    |    |-- metadata: map (nullable = true)
 |    |    |    |-- key: string
 |    |    |    |-- va

In [21]:
result.write.mode("overwrite").parquet(f"{output_path}/patent_keyword_raw.parquet")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Create Transformer to produce keyword and score column

In [22]:
from pyspark import keyword_only
from pyspark.ml import Transformer
from pyspark.ml.param.shared import HasInputCols, HasOutputCol, Param, Params, TypeConverters
from pyspark.ml.util import DefaultParamsReadable, DefaultParamsWritable
from pyspark.sql import DataFrame
from pyspark.sql.types import StringType, ArrayType, StructType, StructField, IntegerType,FloatType, MapType
import pyspark.sql.functions as F

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [24]:
class GenerateKeywordAndScores(Transformer, DefaultParamsReadable, DefaultParamsWritable):
  
  @keyword_only
  def __init__(self):
    super(GenerateKeywordAndScores, self).__init__()
    kwargs = self._input_kwargs
    self.set_params(**kwargs)
    
  @keyword_only
  def set_params(self):
    kwargs = self._input_kwargs
    self._set(**kwargs)
  
  def _transform(self, df: DataFrame):
    df = df.selectExpr("id", "explode(arrays_zip(keywords.result, keywords.metadata)) as resultTuples").selectExpr("id", "resultTuples['0'] as keyword", "resultTuples['1'].score as score").dropDuplicates(["id", "keyword"])
    
    window = w.Window.partitionBy("id").orderBy(F.col("score").asc())
    
    return df.withColumn("row", F.row_number().over(window)).filter(F.col("row")<=10)
    

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [25]:
generate_keyword_and_scores = GenerateKeywordAndScores()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [26]:
pipeline2 = Pipeline().setStages([generate_keyword_and_scores])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [28]:
patent_keywords_path = "s3://patents-analytics/cleaned/patent_keyword_raw.parquet/"

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [29]:
patent_keywords = spark.read.parquet(patent_keywords_path)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [32]:
result2 = pipeline2.fit(patent_keywords).transform(patent_keywords)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [33]:
result2.write.mode("overwrite").parquet(f"{output_path}/patent_keyword.parquet")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [34]:
results3 = spark.read.parquet(f"{output_path}/patent_keyword.parquet")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [36]:
results3.show(100)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+--------------------+--------------------+---+
|      id|             keyword|               score|row|
+--------+--------------------+--------------------+---+
|10000057|              nozzle| 0.03952371456224172|  1|
|10000057|          nozzle set| 0.04459956883003323|  2|
|10000057|        first nozzle| 0.05599600795653027|  3|
|10000057|               print| 0.06424957298731497|  4|
|10000057|                 set| 0.07817739344882932|  5|
|10000057|               first| 0.08385369103345215|  6|
|10000057|          print head| 0.10423852394624489|  7|
|10000057|            printing| 0.11861798711063412|  8|
|10000057|         information| 0.13331647151157514|  9|
|10000057|                head| 0.13331647151157514| 10|
|10000517|           complexes| 0.40392520545533317|  1|
|10000638|              weight| 0.04450358299523055|  1|
|10000638|             asphalt| 0.05620439654996074|  2|
|10000638| asphalt composition|  0.0589761706377734|  3|
|10000638|   synthetic rubbers|